# Kaggle LLM Science Exam Notebook from Thomas Gamet, Apache License 2.0.
 - My aim is to learn techniques and then how to extend them.
 - Many thanks to the fork's original owner Dr. Alvinleenh.
 - I have also learned from Chris Deotte's model and notebook 60k-data-with-context-v2 on which ITK8191 (using DeepSpeed) and ZMSHUM also built and was then studied by me (incorporated into this notebook.)
 - The code also builds on MB's work in Notebook [86.2] with only 270K articles!
 - nagupv/longformer-large-4096-answering-race_LLMMDLPREFOLD_60k_24_09_2023_3 from Hugging Face is also included which provides the 7th and in isolation most accurate model.

Historical notes as experiments were run:
1. Adding "clever" context seems to harm the score.
2. The embeddings need to remain Flat (pretty much original brute force)
3. The predictions in the ensemble have differing means and variances or ranges
4. The model making predictions4 is the worst performing, trying to exclude it (score dropped to .743). Bottom line, this worst of the pack still helps the pack.
5. Adding a statistical weighting to model predictions scored .755
6. Using a majority 3 or 4 of 4 voter, else use ensemble average, scored .759
7. Adding a context change brought the score to .788
8. First replacing model2 with llm-science-run-context-2, score is .819
9. Using matching tokenizers except where rejected and more ensemble tweeks
10. Moved to three model (*context-2 repeated doubling its weight) .825
11. Replaced one *context-2 with another model and maj voter on lowered .819
12. Running with batch size of 8 (was 32) and max_lenth 512 (was 386) .816
13. Back to batch size 32 and for safety also max length 386
14. Running with weights adjusted and majority vote only on full majority 0.826
15. Minor tweeks, BATCH_SIZE 16 and NUM_SENTENCES 10 (maybe 22 is too many)?
16. Run scaling1 = 0.3 scaling2 = 0.1 scaling3 = 0.1 scaling4 = 0.5 scored .821
17. scaling1 = 0.5 caling2 = 0.3 caling3 = 0.3 caling4 = 0.1 scored .827
17. .1 Scaled above similar to scale in .01 drops in training model
18. Confirm if tokenizer is better from each model or from best model scored 
18. Add a model, assess training MAP@3 and update scales to see if score change
18. 1 Adding the model harmed results - voter 4/5 majority scored .826
18. 2 A 3/4/5 majority voter scored .814
18. 3 Just a 5 out of 5 majority vote on new ensemble based on stddev and mean scored .831
18. 4 Trying just the top three models, same technique, reduced to .828
19. New tuning algorithm for the ensemble on training data is producing suboptimal results, and full majority voting 3,4,5 is yielding .817

-- shift every model's predictions to 0 via subtracting its mean
-- magnify each model's output by the number of times its stddev is smaller than the largest standard deviation (the standard deviation is at scale - magnification 1
-- multiple the model predictions by the traing test run on accuracy
-- Further add a scaling weight based on the differences of the accuracies as the scaling factor (someone manual final consideration).

20. Apply the ensemble scaling by testing the public score of each model separately and then applying this to the factors above.
21. Trying two new models (new model 4 and 5) both of which individually score much higher on the Public test set than those they are replacing. However, training scores and Cohen's Kappa therein dropped and this had me concerned. Fortunately, it appears the public data set score is 0.846 suggesting it truly helps the MAP@3 scoring results (the Cohen's Kappa looks only at the first answer being correct and it is really about disagrement negative and agreement positive). 
22. Put "/kaggle/input/how-to-train-open-book-model-part-1/model_v2" back into the ensemble, but don't let it participate in a vote (just have it in the ensemble). This produced a .846 also and likely just a little higher as the version too the new high score.
23. Attempting to weight the top to as 3x, middle 3 as 2x, and bottom scoring as 1x produced a score of .841. Back to scaling all 2x and trying 4 and 5 of 5 majority voting also produced .841.
24. The first attempt to use DeepSpeed v2 with the -base mode and batch size 4 (instead of 2) as model 6 reducedthe score to .841 (all else was producing .846)
25. The second attempt parameterized DeepSpeed as it came, but provided a context2.cvs containing only sources 6 through 12 (7 in all) with bad data removed. Its training score is 0.843 and its public score is 0.767 with an ensemble contribution of 0.838.
26. One test run to see what over-writes by a majority voter does to the public score of 0.836. This means a better model will need to be built and added to the ensemble.
27. Oddly, running with the Gibberish trained model 6 (DeepSpeed v2), but itself underwhelm with a score of 0.36, the full participation in the ensemble let the ensemble still score a 0.845 almost as if it wasn't even there. Still, it is not helping so I'll see if I can find another approach to add.
28. I tried MB's notebook "[86.2] with only 270k articles!" and incorporated its use of a LongFormer to build the predictions of model6. My main concern is that the RAM may not be enough for the Public or Private dataset sizes as it has to keep the Deberta predictions in memory until it has the LongFormer model's work also completed. It's going to be tight on RAM and approximately double the GPU use per experiment. However, the training data self-check is encouraging. The final score ranked Silver at 0.887. 
29. Going to see if a unanimous majority vote helps tip in either direction (a simple majority obviously reinstates the status of the Deberta based models and with 5 of them they win resulting in their lower score). Meanwhile, I'm think about how the longformer can be used to build a second model maybe with Chris' context2. It did not help, the public score stands at 0.887 (no change).
30. As I think through options to add a second LongFormer model to the ensemble I will also try just giving the current LongFormer the weight of 2 models to see what impact that has on the public score (it turns out to be 0.885 so not a good idea).
31. The first alternative LongFormer was from the Kaggle dataset "longformer" which loads with warnings about uninitialized weights and on its own scored 0.816 on the public set, however, when run with the ensemble it brought down the score to 0.82 and so I'm not going to proceed with it.
31. I'm adding another LongFormer (nagupv/longformer-large-4096-answering-race_LLMMDLPREFOLD_60k_24_09_2023_3 from Hugging Face) - it does not affect the training internal score in any significant way, but then that score is pretty close to maxed out so we need to see the Public score which turns out being 0.885). Also, a parallel test confirms the new LongFormer (which combined Chris' data with the race dataset on training) achieves 0.882 on its own).
32. The LongFormer models use softmax which limits the predictions to output no greater than 1 on each answer, and no more than a sum of 1 for each questions answers. This makes comparison to the Deberta models unpredictable which is now resolved by having the Deberta models predictions also put through the softmax algorithm (this preserves the answers the individual models will produce, and scales them to be arithmatically in the same range as each other and the LongFormer model predictions.
33. Weight the two LongFormers heaviest, then the three middle models per known Public score accuracy to be just a little less than the LongFormers together (to break ties among the LongFormers or upset them as long as all three of the top Deberta models agree), and weight the bottom two Deberta models just enough to match a single of the top three Deberta models so long as they both agree. This scored a Public ensemble 0.897
34. Ensemble effectiveness and weights (scaling) study. Experiment 12 will try swapping out "llm-se-debertav3-large" for "how-to-train-open-book-model-part-1/model_v2" in Model 3. Experiment 13 will swap out "deberta-v3-large-freeze16-512/deberta_v3_large_freeze16_512" for "how-to-train-open-book-model-part1/model_v2". The model being swapped out individually underperforms, however, how it is built is different than the other deberta models and longformer so the experiment is to see if helps where those models fail in breaking ties.

Experiment 1-6
Model 1       0.50     1.00     0.821    0.931    0.50     0.60
Model 2       0.25     1.00     0.775    0.879    0.33     0.20
Model 3       0.25     1.00     0.789    0.895    0.33     0.20
Model 4       0.50     1.00     0.822    0.932    0.50     0.60
Model 5       0.50     1.00     0.800    0.907    0.44     0.40
Model 6       1.00     1.00     0.862    0.977    1.00     1.00
Model 7       1.20     1.00     0.882    1.000    1.20     1.20
Trn Score     0.983    0.985    0.985    0.985    0.983    0.983
Pub Score     0.897    0.884    0.888    0.888    0.895    0.985
Cohen Kappa   0.9497   0.9560   0.9560   0.9560   0.9497   0.9497

Experiment 7-13        v84      v85      v87      v88      v89      v90
Model 1       0.52     0.42     0.50     0.33     0.60     0.60     0.60
Model 2       0.32     0.42     0.25     0.20     0.20     0.20     0.20
Model 3       0.32     0.42     0.25     0.20*    0.20     0.20     0.20
Model 4       0.52     0.42     0.50     0.33     0.60*    0.60     0.60
Model 5       0.42     0.42     0.33     0.33     0.40     0.40*    0.40*
Model 6       1.00     1.00     1.00     1.00     1.00     1.00     1.00
Model 7       1.10     1.10     1.00     1.20     1.30     1.30     1.30
Trn Score     0.983    0.983    0.983    0.982    0.983    0.985    0.983
Pub Score     0.895    0.895    0.895    0.895    0.897    0.893    0.893
Cohen Kappa   0.9497   0.9497   0.9497   0.9497   0.9497   0.9560   0.9497

Experiment 14 v91      v92      v93      v94
Model 1       0.60     0.50     0.50     0.50
Model 2       0.20*    0.25*    0.50*    0.20*
Model 3       0.20     0.25     0.50     0.00
Model 4       0.60     0.50     0.50     0.50
Model 5       0.40     0.50     0.50     0.40
Model 6       1.00     1.00     1.00     1.00
Model 7       1.20     1.20     1.20     1.20
Trn Score     0.983    0.983    0.987    0.985
Pub Score     0.895    0.897    0.894    0.894
Cohen Kappa   0.9497   0.9497   0.9560   0.9560

* is after a model swap to put openbook into the ensemble
+ Options are exhausted on weights in the ensemble - it needs a new model

35. Using v92 weights, instructing the LongFormer version to use their probabilities on values > 0.3 (instead of 0.4) to see what difference this makes in v95. Trn Score 0.983, Cohen Kappa 0.9497, Pub Score 0.897
36. Run with the threshold at 0.00 from above - I am having my doubts about it.  Also set the Deberta model wiki context to a MAX_LENGTH of 512 and BATCH_SIZE of 1 hoping this helps with generalization. Trn Score 0.983, Cohen Kappa 0.9497, Pub Score 0.898
37. Out of concern that the submission_backup is only 200 elements long I want to be ssure it is not possible to be used with the private data set so I will replicate the results with the LongFormer unable to use it while leaving a trace of the decision. The difference between the two planned entries is one model (so this will be a little more - it will include the change to BATCH_SIZE and MAX_LENGTH too.

# Install faiss, sentence-transformer, and blingfire
We are using PyTorch

In [1]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126134 sha256=94e0f04e24c6cef6e9ea52fbe220a16b8b66a147703cf7be7c8d07b7e52721ba
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl
Processing /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Processing /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
Processing /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
  Attempting

# Critical Imports

In [2]:
# We will be using PyTorch and a set of functions that consistently
# stage Training for Multiple Choice problems using Facebook faiss
# and a single Tokenizer that works with Deberta models. The embbeddings
# are Flat.
#
# Deberta remains chosen as a generally best results producing BERT 
# model which can be trained in advance.
#
# Two LongFormer models are also used before the ensemble is built though
# they are built separately and before they can be built the deberta models
# need to be deleted with their memory collected.

import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations
from collections.abc import Iterable
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")
from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# Emulate a run where the Private data set needs to have its results retained.
# Each Private prediction will take about 10MB and we're keeping 5-6 of them.
shapetoreserve = (6*4000, 5) # this leaves a few higher level structures out
predictionsizetoreserve = np.zeros(shapetoreserve)

# Set up functions to process documents and tokenize sentences

In [3]:
# Three functions are defined to process sentences to be read from the wiki
# documents to be used in training. The wiki document data is
# passed throught these document/sentence functions to split into tokens
# and put through the tokenizer.

def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:

    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df

def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)
        processed_documents.append(row)
    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df

def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [4]:
# Before we can use the functions above a few configuration values
# are named and assigned values

SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0 # The GPU
MAX_LENGTH = 512 # since the batch is smaller let's try a bigger length too than 386 # better than when using 512
BATCH_SIZE = 1 # trying smaller batch size one more time since this often helps with generalization 16 # back to original  # performed better than a size of 8
WIKI_PATH = "/kaggle/input/wikipedia-20230701" # source of data the huge LLM used to generate the multiple choice test
wiki_files = os.listdir(WIKI_PATH)

# Setup the training data for training loop use

In [5]:
# read the wiki documents and produce a CSV file for per the final values
# let in DataFrame trn.

trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)
## Search using the prompt and answers to guide the search
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

model2 = SentenceTransformer(SIM_MODEL, device=torch.device('cuda'))
model2.max_seq_length = MAX_LENGTH
model2 = model2.half()
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")
#prompt_embeddings = model2.encode(trn.prompt.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = model2.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()
#search_score, search_index = sentence_index.search(prompt_embeddings, 3)
search_score, search_index = sentence_index.search(prompt_embeddings, 6)
del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])
## keeping a note, it is used in /kaggle/input/llm-science-run-context-2
## you won't be training with the data here - here we're just creating
## a vector store of the wiki documents as embeddings
#df = pd.read_csv("/kaggle/input/60k-data-with-context-v2/train_with_context2.csv")
## Get the article and associated file location using the index
wikipedia_file_data = []

for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

## Save memory - delete df since it is no longer necessary
del df
_ = gc.collect()
libc.malloc_trim(0)

## Get the full text data
wiki_text_data = []

for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

## Parse documents into sentences
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)

## Get embeddings of the wiki text data
wiki_data_embeddings = model2.encode(processed_wiki_text_data.text,
                                    batch_size=BATCH_SIZE,
                                    device=DEVICE,
                                    show_progress_bar=True,
                                    convert_to_tensor=True,
                                    normalize_embeddings=True)#.half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()
print("The wiki_data_embeddings has",wiki_data_embeddings.shape[1],"features.")
print("The wiki_data_embeddings has",wiki_data_embeddings.shape[0],"rows of those features")
_ = gc.collect()

## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)

## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

question_embeddings = model2.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
question_embeddings = question_embeddings.detach().cpu().numpy()

#NUM_SENTENCES_INCLUDE = 3
# this may be further tuned - 3 was outdone by 20, but 10 outdid 20 and presumably 22
NUM_SENTENCES_INCLUDE = 10 # was 22, lets see maybe too many is possible - yip 10 beats 20 comparing model 25 to model 26

## List containing just Context
contexts = []

for r in tqdm(trn.itertuples(), total=len(trn)):

    prompt_id = r.Index

    prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values)].index.values

    if prompt_indices.shape[0] > 0: # started with only Flat
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        prompt_index.add(wiki_data_embeddings[prompt_indices])

        # context = "Look at the best results for each question at least three times focusing on the scientific consensus. "
        # Adding "context" causes a lower score.
        context = ""
        
        ## Get the top matches
        ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
            context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
        
    contexts.append(context)
    
trn['context'] = contexts
trn[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)

/tmp/ipykernel_23/3637232308.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)


Batches:   0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/1120 [00:00<?, ?it/s]

Batches:   0%|          | 0/64266 [00:00<?, ?it/s]

The wiki_data_embeddings has 384 features.
The wiki_data_embeddings has 64266 rows of those features


Batches:   0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

# Set up Training batch processing and processing of answer/index

In [6]:
# Below are two functions used during Training of pretrained models
# before asking the model to perform predictions on the Test data.
# There are also two Dictionaries to convert index 1 to 5 to options A to E.

# The data class DataCollatorForMultipleChoice has a constructor that 
# flattens and tokenizes one batch at a time.

def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        # lots of tokens, two types token ids (0 and 1), and 1 attention mask for token ids (1)
        # print("There are",flattened_features,"for label",label_name)
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

# Train and run Predictions for all defined Deberta models
Each model is defined at the top of its own training and prediction code block

In [7]:
# tmodel = "/kaggle/input/2023kagglellm-deberta-v3-large-model1"
tmodel = "/kaggle/input/llm-science-run-context-2"
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df.id = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[:1500]) + " #### " +  test_df["prompt"] # made 1500 into 2000
test_df['answer'] = 'A'
test_ds = Dataset.from_pandas(test_df)
tokenizer = AutoTokenizer.from_pretrained(tmodel)

model = AutoModelForMultipleChoice.from_pretrained(tmodel) #"/kaggle/input/deberta-llm-science-074/model_3")    

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
)

tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
test_predictions = trainer.predict(tokenized_test_ds)
#print(test_predictions)

/tmp/ipykernel_23/3493626086.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  test_df.id = list(range(len(test_df)))


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [8]:
tmodel = "/kaggle/input/2023kagglellm-deberta-v3-large-model1" # back 10/7/2023 out 10/4/2023
#tmodel = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2"
#tmodel = "/kaggle/input/deberta-llm-science-074/model_11" do no use
#tokenizer = AutoTokenizer.from_pretrained(tmodel)
model = AutoModelForMultipleChoice.from_pretrained(tmodel)    

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
)

tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
test_predictions2 = trainer.predict(tokenized_test_ds)
#print(test_predictions2)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
#tmodel = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2"
tmodel = "/kaggle/input/llm-se-debertav3-large"
#tokenizer = AutoTokenizer.from_pretrained(tmodel)
model = AutoModelForMultipleChoice.from_pretrained(tmodel)    

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
)

tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
test_predictions3 = trainer.predict(tokenized_test_ds)
#print(test_predictions3)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
tmodel = "/kaggle/input/deberta-v3-large-freeze16-512/deberta_v3_large_freeze16_512" # may be worth trying
#tmodel = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2"
#tokenizer = AutoTokenizer.from_pretrained(tmodel)
model = AutoModelForMultipleChoice.from_pretrained(tmodel)  

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
)

tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
test_predictions4 = trainer.predict(tokenized_test_ds)
#print(test_predictions4)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
tmodel = "/kaggle/input/using-deepspeed-with-hf-trainer/checkpoints_1" # may be worth trying
#tmodel = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2"
#tokenizer = AutoTokenizer.from_pretrained(tmodel) # must use another's tokenizer
model = AutoModelForMultipleChoice.from_pretrained(tmodel)  

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
)

tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
test_predictions5 = trainer.predict(tokenized_test_ds)
#print(test_predictions4)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
# reclaim as much memory as possible for what follows rebuilds what it needs
# the code above produced Deberta based predictions
# the code below uses a LongFormer to make predictions

del model
del trainer
del wikipedia_file_data
del wiki_text_data
del processed_wiki_text_data
del question_embeddings
del context
del model2
del wiki_data_embeddings
del trn
del test_df
del test_ds
del tmodel
del tokenized_test_ds
_ = gc.collect()
libc.malloc_trim(0)

1

# New type of model learned from MB
I am studying and reusing MB's work in Notebook [86.2] with only 270K articles!

In [13]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
!cp /kaggle/input/backup-806/util_openbook.py .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.3
    Uninstalling datasets-2.14.3:
      Successfully uninstalled datasets-2.14.3
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks.

In [14]:
!cp /kaggle/input/backup-806/util_openbook.py .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Leverage util_openbook

The code block below consumes nearly all the available memory. For now,
the code fits and the two print statements showing entry and exit are left.

In [15]:
from util_openbook import get_contexts, generate_openbook_output
import pickle

print("Working on context and generating openbook_output")
get_contexts()
generate_openbook_output()

import gc
gc.collect()
print("We made it! We have run the generate openbook output!")

Working on context and generating openbook_output


/kaggle/working/util_openbook.py:137: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


We made it! We have run the generate openbook output!


# The code block below defines functions that will read the Wiki Docs
They process the data from the Wiki documents into sentence that are then
tokenized. They also produce vectorized vocabulary which has its length printed.

In [16]:
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer

def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

def get_relevant_documents_parsed(df_valid):
    df_chunk_size=600
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles

def get_relevant_documents(df_valid):
    df_chunk_size=800
    
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles

def retrieval(df_valid, modified_texts):
    
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words)
    vectorizer1.fit(corpus_df_valid)
    vocab_df_valid = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid)
    vectorizer.fit(modified_texts[:500000])
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 10
    top_per_query = 10

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores

def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = context + ' ' + tokenizer.bos_token + ' ' + question
    c_plus_q_4 = [c_plus_q] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to(model.device.index),
        "attention_mask": attention_mask.to(model.device.index),
    }
    return example_encoded

In [17]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']

# [Re]Load the test data

In [18]:
df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")

In [19]:
# import pandas as pd
backup_model_predictions = pd.read_csv("submission_backup.csv")
print(backup_model_predictions.head(10))
print(backup_model_predictions.shape)

   id prediction
0   0      D B E
1   1      A B D
2   2      A C D
3   3      C E D
4   4      D A B
5   5      B C E
6   6      A C B
7   7      D E B
8   8      C B A
9   9      A B E
(200, 2)


In [20]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
_ = gc.collect()

retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()
libc.malloc_trim(0)

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 11222


  0%|          | 0/22 [00:00<?, ?it/s]

1

In [22]:
import unicodedata

retrieved_articles = get_relevant_documents(df_valid)
gc.collect()
libc.malloc_trim(0)

Map (num_proc=2):   0%|          | 0/2781652 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 11222


  0%|          | 0/28 [00:00<?, ?it/s]

1

# Turn this to a code block to do some sizing of predictions (private scoring uses 4000ish)
import sys
print(sys.getsizeof(test_predictions5))
print(sys.getsizeof(test_predictions5.predictions))
print(sys.getsizeof(test_predictions5.predictions[0]))
print(sys.getsizeof(test_predictions5.predictions[0][0]))
print ("200 predictions need ",64+128+112*28*200)
print ("600 predictions need ",64+128+112*28*600)
print ("4000 predictions need ",64+128+112*28*4000)
print ("4000 predictions need ",64+128+112*28*4000*6)

In [23]:
from transformers import LongformerTokenizer, LongformerForMultipleChoice

tokenizer = LongformerTokenizer.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model")
model = LongformerForMultipleChoice.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model").cuda()

# Inferencing begins with the LongFormer models
The code retrieves articles related to the multiple choice question to provide context for the prediction.

In [24]:
import torch

test_predictions6 = []
submit_ids = []
shape6 = (df_valid.shape[0], 5)
predictions6 = np.zeros(shape6)
pcount = 0

for index in tqdm(range(df_valid.shape[0])):
    columns = df_valid.iloc[index].values
    submit_ids.append(columns[0])
    question = columns[1]
    options = [columns[2], columns[3], columns[4], columns[5], columns[6]]
    context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"
    context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
    inputs1 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context1,
        )
    inputs2 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context2,
        )
    
    with torch.no_grad():
        outputs1 = model(**inputs1)    
        losses1 = -outputs1.logits[0].detach().cpu().numpy()
        probability1 = torch.softmax(torch.tensor(-losses1), dim=-1)
        
    with torch.no_grad():
        outputs2 = model(**inputs2)
        losses2 = -outputs2.logits[0].detach().cpu().numpy()
        probability2 = torch.softmax(torch.tensor(-losses2), dim=-1)
        
    probability_ = (probability1 + probability2)/2

    if probability_.max() > 0.0:  # was 0.4 as a threshold
        predict = np.array(list("ABCDE"))[np.argsort(probability_)][-3:].tolist()[::-1]
    else:
        predict = backup_model_predictions.iloc[index].prediction.replace(" ","")
    test_predictions6.append(predict)
    for i in range(probability_.shape[0]):
        #print ("Prediction",pcount,"element",i,"value",probability_[i])
        predictions6[pcount][i] = probability_[i].item()
        #print ("Assigned",predictions6[pcount][i])
    pcount += 1

test_predictions6 = [" ".join(i) for i in test_predictions6]

  0%|          | 0/200 [00:00<?, ?it/s]

In [25]:
del tokenizer
del model

gc.collect()
libc.malloc_trim(0)

1

In [26]:
from transformers import LongformerTokenizer, LongformerForMultipleChoice

tokenizer = LongformerTokenizer.from_pretrained("/kaggle/input/longformer-large-race-60k-24-09-2023-3-26")
model = LongformerForMultipleChoice.from_pretrained("/kaggle/input/longformer-large-race-60k-24-09-2023-3-26").cuda()

In [27]:
test_predictions7 = []
submit_ids = []
shape7 = (df_valid.shape[0], 5)
predictions7 = np.zeros(shape7)
pcount = 0

for index in tqdm(range(df_valid.shape[0])):
    columns = df_valid.iloc[index].values
    submit_ids.append(columns[0])
    question = columns[1]
    options = [columns[2], columns[3], columns[4], columns[5], columns[6]]
    context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"
    context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
    inputs1 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context1,
        )
    inputs2 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context2,
        )
    
    with torch.no_grad():
        outputs1 = model(**inputs1)    
        losses1 = -outputs1.logits[0].detach().cpu().numpy()
        probability1 = torch.softmax(torch.tensor(-losses1), dim=-1)
        
    with torch.no_grad():
        outputs2 = model(**inputs2)
        losses2 = -outputs2.logits[0].detach().cpu().numpy()
        probability2 = torch.softmax(torch.tensor(-losses2), dim=-1)
        
    probability_ = (probability1 + probability2)/2

    if probability_.max() > 0.0: # was 0.4
        predict = np.array(list("ABCDE"))[np.argsort(probability_)][-3:].tolist()[::-1]
    else:
        predict = backup_model_predictions.iloc[index].prediction.replace(" ","")
    test_predictions7.append(predict)
    for i in range(probability_.shape[0]):
        #print ("Prediction",pcount,"element",i,"value",probability_[i])
        predictions7[pcount][i] = probability_[i].item()
        #print ("Assigned",predictions7[pcount][i])
    pcount += 1

test_predictions7 = [" ".join(i) for i in test_predictions7]

  0%|          | 0/200 [00:00<?, ?it/s]

In [28]:
_ = gc.collect()
libc.malloc_trim(0)

1

print(predictions6)

# Generate some descriptive statistics for each prediction
I am leaving this for it gives a "feel" for what the individual models produce.

In [29]:
del tokenizer
del model
_ = gc.collect()

In [30]:
# Calculate descriptive statistics
mean1 = np.mean(test_predictions.predictions)  # Mean
median1 = np.median(test_predictions.predictions)  # Median
std_dev1 = np.std(test_predictions.predictions)  # Standard Deviation
variance1 = np.var(test_predictions.predictions)  # Variance
min_value1 = np.min(test_predictions.predictions)  # Minimum Value
max_value1 = np.max(test_predictions.predictions)  # Maximum Value

predictions1 = test_predictions.predictions

#if len(predictions1) == 200:
#    print(predictions1)

# Print the results
print("Predictions1 descriptive statistics:")
print(f"Mean: {mean1}")
print(f"Median: {median1}")
print(f"Standard Deviation: {std_dev1}")
print(f"Variance: {variance1}")
print(f"Minimum Value: {min_value1}")
print(f"Maximum Value: {max_value1}")

Predictions1 descriptive statistics:
Mean: -0.32640209794044495
Median: -0.6024494171142578
Standard Deviation: 4.171014785766602
Variance: 17.39736557006836
Minimum Value: -8.118309020996094
Maximum Value: 8.923480987548828


In [31]:
# Calculate descriptive statistics
mean2 = np.mean(test_predictions2.predictions)  # Mean
median2 = np.median(test_predictions2.predictions)  # Median
std_dev2 = np.std(test_predictions2.predictions)  # Standard Deviation
variance2 = np.var(test_predictions2.predictions)  # Variance
min_value2 = np.min(test_predictions2.predictions)  # Minimum Value
max_value2 = np.max(test_predictions2.predictions)  # Maximum Value

predictions2 = test_predictions2.predictions

# Print the results
print("Predictions2 descriptive statistics:")
print(f"Mean: {mean2}")
print(f"Median: {median2}")
print(f"Standard Deviation: {std_dev2}")
print(f"Variance: {variance2}")
print(f"Minimum Value: {min_value2}")
print(f"Maximum Value: {max_value2}")

Predictions2 descriptive statistics:
Mean: 0.4437539577484131
Median: 0.4372491240501404
Standard Deviation: 0.06565383821725845
Variance: 0.004310426767915487
Minimum Value: 0.24167223274707794
Maximum Value: 0.722373366355896


In [32]:
# Calculate descriptive statistics
mean3 = np.mean(test_predictions3.predictions)  # Mean
median3 = np.median(test_predictions3.predictions)  # Median
std_dev3 = np.std(test_predictions3.predictions)  # Standard Deviation
variance3 = np.var(test_predictions3.predictions)  # Variance
min_value3 = np.min(test_predictions3.predictions)  # Minimum Value
max_value3 = np.max(test_predictions3.predictions)  # Maximum Value

predictions3 = test_predictions3.predictions

# Print the results
print("Predictions3 descriptive statistics:")
print(f"Mean: {mean3}")
print(f"Median: {median3}")
print(f"Standard Deviation: {std_dev3}")
print(f"Variance: {variance3}")
print(f"Minimum Value: {min_value3}")
print(f"Maximum Value: {max_value3}")

Predictions3 descriptive statistics:
Mean: -2.777772903442383
Median: -3.3575916290283203
Standard Deviation: 4.20194149017334
Variance: 17.65631103515625
Minimum Value: -8.663307189941406
Maximum Value: 8.013126373291016


In [33]:
# Calculate descriptive statistics
mean4 = np.mean(test_predictions4.predictions)  # Mean
median4 = np.median(test_predictions4.predictions)  # Median
std_dev4 = np.std(test_predictions4.predictions)  # Standard Deviation
variance4 = np.var(test_predictions4.predictions)  # Variance
min_value4 = np.min(test_predictions4.predictions)  # Minimum Value
max_value4 = np.max(test_predictions4.predictions)  # Maximum Value

predictions4 = test_predictions4.predictions

# Print the results
print("Predictions4 descriptive statistics:")
print(f"Mean: {mean4}")
print(f"Median: {median4}")
print(f"Standard Deviation: {std_dev4}")
print(f"Variance: {variance4}")
print(f"Minimum Value: {min_value4}")
print(f"Maximum Value: {max_value4}")

Predictions4 descriptive statistics:
Mean: -3.445631265640259
Median: -4.980574131011963
Standard Deviation: 6.223476886749268
Variance: 38.73166275024414
Minimum Value: -10.93929672241211
Maximum Value: 11.748093605041504


In [34]:
# Calculate descriptive statistics
mean5 = np.mean(test_predictions5.predictions)  # Mean
median5 = np.median(test_predictions5.predictions)  # Median
std_dev5 = np.std(test_predictions5.predictions)  # Standard Deviation
variance5 = np.var(test_predictions5.predictions)  # Variance
min_value5 = np.min(test_predictions5.predictions)  # Minimum Value
max_value5 = np.max(test_predictions5.predictions)  # Maximum Value

predictions5 = test_predictions5.predictions

# Print the results
print("Predictions5 descriptive statistics:")
print(f"Mean: {mean5}")
print(f"Median: {median5}")
print(f"Standard Deviation: {std_dev5}")
print(f"Variance: {variance5}")
print(f"Minimum Value: {min_value5}")
print(f"Maximum Value: {max_value5}")

Predictions5 descriptive statistics:
Mean: 0.017698463052511215
Median: 0.010806635022163391
Standard Deviation: 2.7343733310699463
Variance: 7.476798057556152
Minimum Value: -7.5592875480651855
Maximum Value: 7.109035491943359


In [35]:
# Calculate descriptive statistics
#predictions6 = np.array(test_predictions6)
# these work as a softmax percentile, no need to clip "outliers"

mean6 = np.mean(predictions6)  # Mean
median6 = np.median(predictions6)  # Median
std_dev6 = np.std(predictions6)  # Standard Deviation
variance6 = np.var(predictions6)  # Variance
min_value6 = np.min(predictions6)  # Minimum Value
max_value6 = np.max(predictions6)  # Maximum Value

# Print the results
print("Predictions6 descriptive statistics:")
print(f"Mean: {mean6}")
print(f"Median: {median6}")
print(f"Standard Deviation: {std_dev6}")
print(f"Variance: {variance6}")
print(f"Minimum Value: {min_value6}")
print(f"Maximum Value: {max_value6}")

Predictions6 descriptive statistics:
Mean: 0.19999999966516566
Median: 0.04470626637339592
Standard Deviation: 0.30263246041871755
Variance: 0.09158640609908664
Minimum Value: 3.047210611839546e-06
Maximum Value: 0.9999765753746033


In [36]:
# Calculate descriptive statistics
#predictions6 = np.array(test_predictions6)
# these work as a softmax percentile, no need to clip "outliers"

mean7 = np.mean(predictions7)  # Mean
median7 = np.median(predictions7)  # Median
std_dev7 = np.std(predictions7)  # Standard Deviation
variance7 = np.var(predictions7)  # Variance
min_value7 = np.min(predictions7)  # Minimum Value
max_value7 = np.max(predictions7)  # Maximum Value

# Print the results
print("Predictions7 descriptive statistics:")
print(f"Mean: {mean7}")
print(f"Median: {median7}")
print(f"Standard Deviation: {std_dev7}")
print(f"Variance: {variance7}")
print(f"Minimum Value: {min_value7}")
print(f"Maximum Value: {max_value7}")

Predictions7 descriptive statistics:
Mean: 0.19999999911600116
Median: 0.03429394215345383
Standard Deviation: 0.3175730911887547
Variance: 0.10085266824718113
Minimum Value: 2.5518775146338157e-06
Maximum Value: 0.999972939491272


# Re-scale predictions so their numeric range is weighted more by their accuracy than by their prediction's statistical distribution

In [37]:
# With the LongFormer introducing softmax based results on the 5 answers I
# I am changing the way the Deberta output works by taking the softmax of
# each of the 5 answers to every question presented to them. This should not 
# change their public score since the public score of each individual run is
# based upon their relative order which is unchanged under a softmax transformation.

# However, each model in the ensemble is not equally good at making predictions
# Also, even though softmax guarantees each set of answers adds up to 1.00 
# probability, and no single entry is more than 1.00 either, the models
# differ by how likely they are to be highly confident or less confident in
# determining their top pick.

scaling1 = 0.50 # Pub Score 0.821 -- mid level - break ties between 6 and 7
scaling2 = 0.25 # Pub Score 0.775 -- works at the bottom to break 1,4 and 5 ties
scaling3 = 0.25 # Pub Score 0.789 -- works at the bottom to break 1,4 and 5 ties
scaling4 = 0.50 # Pub Score 0.822 -- mid level - break ties between 6 and 7
scaling5 = 0.50 # Pub Score 0.800 -- mid level - break ties between 6 and 7
scaling6 = 1.00 # Pub Score 0.862 -- will overpower scaling7
scaling7 = 1.20 # Pub Score 0.882 -- must be a little bigger than scaling6

# OLD NOTES from before adding the LongFormer models

## Score for models on training and public data when run in isolation 9/16/23
# Model 1: Training Score 0.945 and Public Score 0.821 drop by 0.124
# Model 2: Training Score 0.913 and Public Score 0.775 drop by 0.138
# Model 3: Training Score 0.885 and Public Score 0.789 drop by 0.096
# Model 4: Training Score 0.812 and Public Score 0.717 drop by 0.095
# Model 4: Training Score 0.897 and Public Score 0.822 drop by 0.075 9/18/23
# Model 5: Training Score 0.807 and Public Score 0.620 drop by 0.250
# Model 5: Training Score 0.885 and Public Score 0.800 drop by 0.085 9/17/23

# there's a good chance we'll want both model4 candidates in the ensemble, but first run some tests
#Model 4 with .710 tmodel = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2"
#Model 4 with .897 tmodel = "/kaggle/input/deberta-v3-large-freeze16-512/deberta_v3_large_freeze16_512"

#Model 5 with .885 tmodel = "/kaggle/input/using-deepspeed-with-hf-trainer/checkpoints_1"
#Model 5 with .620 tmodel = "/kaggle/input/deberta-llm-science-074/model_13"

#Model 6: Training Score 0.863 and Public Score 0.758 -> ensemble scored 0.841
# Attempting to build with Deberta-large, batch size 2, and only the sciency
# sections in the context2 with four answer cases fixed or removed (cleaned up).
# DeepSpeed (using Deberta large) yeilded
#Model 6: Training Score 0.843 and Public Score 0.767 -> ensemble may be 0.838


# Generate the ensemble after applying softmax to Deberta model predictions

In [38]:
# For the deberta models, calculate the softmax for each row's answers
# We need this to make sure the ensemble is scaled and usable
def rowsoftmax(data):
    for i in range(len(data)):
        exp_x = np.exp(data[i,:])
        sum_exp_x = np.sum(exp_x)
        data[i,:] = exp_x / sum_exp_x
    return data

In [39]:
tt1 = rowsoftmax(predictions1) # Various ranges and means - convert to softmax 
tt2 = rowsoftmax(predictions2) # Various ranges and means - convert to softmax 
tt3 = rowsoftmax(predictions3) # Various ranges and means - convert to softmax 
tt4 = rowsoftmax(predictions4) # Various ranges and means - convert to softmax 
tt5 = rowsoftmax(predictions5) # Various ranges and means - convert to softmax
tt6 = predictions6 # LongFormer returns softmax range 0 to 1
tt7 = predictions7 # LongFormer returns softmax range 0 to 1

# The commented out prediction method was used to combine just Deberta predictions
# prediction = scaling1*(tt1-mean1)+scaling2*(tt2-mean2)+scaling3*(tt3-mean3)+scaling4*(tt4-mean4)+scaling5*(tt5-mean5)+scaling6*(tt6-mean6)+scaling7*(tt7-mean7)

# The new approach has all predictions made as consistent as possible with softmax
prediction = scaling1*tt1+scaling2*tt2+scaling3*tt3+scaling4*tt4+scaling5*tt5+scaling6*tt6+scaling7*tt7

# Use the predictions to fill out the submission dataframe.
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df.id = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[:1500]) + " #### " +  test_df["prompt"] # made 1500 into 2000
test_df['answer'] = 'A'
test_ds = Dataset.from_pandas(test_df)
submission = pd.DataFrame({"id": np.arange(len(test_df))})
submission['prediction'] = predictions_to_map_output(prediction)

/tmp/ipykernel_23/432880219.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  test_df.id = list(range(len(test_df)))


# Generate the predictions for each of the models separately (what they would produce on their own)

In [40]:
prediction = 0.1*test_predictions.predictions
submission1 = pd.DataFrame({"id": np.arange(len(test_df))})
submission1['prediction'] = predictions_to_map_output(prediction)
print(submission1.tail(20))
print(submission1.shape)

      id prediction
180  180      B A E
181  181      A D C
182  182      A C E
183  183      C D A
184  184      A B E
185  185      A E C
186  186      C A D
187  187      A E D
188  188      D C B
189  189      B C E
190  190      E D C
191  191      C D E
192  192      B A E
193  193      B C D
194  194      D B E
195  195      C E A
196  196      B C A
197  197      B A D
198  198      D C A
199  199      A C D
(200, 2)


In [41]:
prediction = 0.1*test_predictions2.predictions
submission2 = pd.DataFrame({"id": np.arange(len(test_df))})
submission2['prediction'] = predictions_to_map_output(prediction)
print(submission2.tail(20))
print(submission2.shape)

      id prediction
180  180      B E A
181  181      A C E
182  182      A C D
183  183      C A D
184  184      A B E
185  185      A C E
186  186      C D A
187  187      A D B
188  188      D B C
189  189      B E D
190  190      E C D
191  191      C E B
192  192      A B E
193  193      B D A
194  194      D B E
195  195      C E A
196  196      B C A
197  197      B A E
198  198      A D B
199  199      C A D
(200, 2)


In [42]:
prediction = 0.1*test_predictions3.predictions
submission3 = pd.DataFrame({"id": np.arange(len(test_df))})
submission3['prediction'] = predictions_to_map_output(prediction)
print(submission3.tail(20))
print(submission3.shape)

      id prediction
180  180      B A E
181  181      A C D
182  182      A C B
183  183      C A D
184  184      A B E
185  185      E A C
186  186      C D A
187  187      C B D
188  188      C B D
189  189      B C D
190  190      C D E
191  191      E D C
192  192      B D C
193  193      B D C
194  194      B D E
195  195      C A E
196  196      C B A
197  197      B A E
198  198      D C B
199  199      A E C
(200, 2)


In [43]:
prediction = 0.1*test_predictions4.predictions
submission4 = pd.DataFrame({"id": np.arange(len(test_df))})
submission4['prediction'] = predictions_to_map_output(prediction)
print(submission4.tail(20))
print(submission4.shape)

      id prediction
180  180      B A E
181  181      A C E
182  182      A C D
183  183      C A D
184  184      A B E
185  185      E A D
186  186      C A E
187  187      D A E
188  188      E D A
189  189      B C E
190  190      E D C
191  191      C E D
192  192      B E D
193  193      B D C
194  194      D B E
195  195      C A E
196  196      C A B
197  197      B A E
198  198      A D C
199  199      A B E
(200, 2)


In [44]:
prediction = 0.1*test_predictions5.predictions
submission5 = pd.DataFrame({"id": np.arange(len(test_df))})
submission5['prediction'] = predictions_to_map_output(prediction)
print(submission5.tail(20))
print(submission5.shape)

      id prediction
180  180      B A D
181  181      A E C
182  182      A C E
183  183      C A B
184  184      A B E
185  185      A E D
186  186      C A E
187  187      D A E
188  188      C B D
189  189      B C E
190  190      E D A
191  191      C D E
192  192      B E A
193  193      B D A
194  194      B D E
195  195      C A E
196  196      A D C
197  197      B A E
198  198      D C B
199  199      A D E
(200, 2)


In [45]:
prediction = 0.1*predictions6
submission6 = pd.DataFrame({"id": np.arange(len(test_df))})
submission6['prediction'] = predictions_to_map_output(prediction)
print(submission6.tail(20))
print(submission6.shape)

      id prediction
180  180      B A C
181  181      A C D
182  182      A E D
183  183      C D A
184  184      A B D
185  185      A E D
186  186      C E D
187  187      A E D
188  188      D C B
189  189      B A E
190  190      E C D
191  191      B D C
192  192      B D E
193  193      B D C
194  194      D E C
195  195      C B D
196  196      B C A
197  197      A B D
198  198      D C B
199  199      C A E
(200, 2)


In [46]:
prediction = 0.1*predictions7
submission7 = pd.DataFrame({"id": np.arange(len(test_df))})
submission7['prediction'] = predictions_to_map_output(prediction)
print(submission7.tail(20))
print(submission7.shape)

      id prediction
180  180      B A C
181  181      A C D
182  182      A E D
183  183      C D B
184  184      A B D
185  185      A E D
186  186      C E B
187  187      A D E
188  188      D C B
189  189      B A E
190  190      E D C
191  191      C D E
192  192      B D E
193  193      B D C
194  194      D E C
195  195      C A E
196  196      B C E
197  197      B A D
198  198      D C B
199  199      C E A
(200, 2)


# The following code was used to see how well a majority voter works 
The ensemble output provides the base, but majority voting can be used
to over-ride it.
Experiments showed that majority voter is less accurate or less likely
to score high.

# This is another option which for now is not being used as the public
# and training datasets suggest it will produce lower scores than the
# adding up of softmax results, weighted as shown above, and then choosing
# the three answers based upon their softmax totals (which is also a kind
# of majority vote.)

for i in range(len(submission)):
    # The first answer majority vote
    anscount = [0, 0, 0, 0, 0]
    theletter = ["A","B","C","D","E"]
    idx = ord(submission1.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission2.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission3.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission4.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission5.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission6.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    idx = ord(submission7.loc[i,'prediction'][0])-ord("A")
    anscount[idx] = anscount[idx]+1
    #for j in range(5): # the majority voter seems to only help a little and often only on a unanimous vote, but results are mixed
        #if anscount[j] == 4: # we have a simple majority at index j
        #    submission.loc[i,'prediction'] = theletter[j]+submission.loc[i,'prediction'][1:5]
        #if anscount[j] == 5: # we have a major majority at index j
        #    submission.loc[i,'prediction'] = theletter[j]+submission.loc[i,'prediction'][1:5]
        #if anscount[j] == 6: # we have a unanimous majority at index j
        #    submission.loc[i,'prediction'] = theletter[j]+submission.loc[i,'prediction'][1:5]
        #if anscount[j] == 7: # we have a unanimous majority at index j
        #    submission.loc[i,'prediction'] = theletter[j]+submission.loc[i,'prediction'][1:5]



# Write the CSV file with the submission

In [47]:
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').tail(20)

,id,prediction
180,180,B A D
181,181,A C D
182,182,A E C
183,183,C D A
184,184,A B D
185,185,A E D
186,186,C E A
187,187,A E D
188,188,D C B
189,189,B A C


# Below here we run a MAP@3 estimate of each model, the ensemble, and a Cohen Kappa of the predictions agreement - all using just the training data
In general, and as expected, the Public score is usually lower than the Training score and in similar proportions. An inversion would also signal over-fitting.

In [48]:
train_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv").drop("id", 1)
print(train_df.tail())

                                                prompt  \
195  What is the relation between the three moment ...   
196  What is the throttling process, and why is it ...   
197  What happens to excess base metal as a solutio...   
198  What is the relationship between mass, force, ...   
199  What did Arthur Eddington discover about two o...   

                                                     A  \
195  The three moment theorem expresses the relatio...   
196  The throttling process is a steady flow of a f...   
197  The excess base metal will often solidify, bec...   
198  Mass is a property that determines the weight ...   
199  Arthur Eddington showed that two of Einstein's...   

                                                     B  \
195  The three moment theorem is used to calculate ...   
196  The throttling process is a steady adiabatic f...   
197  The excess base metal will often crystallize-o...   
198  Mass is an inertial property that determines a...   
199  Arthur 

/tmp/ipykernel_23/742789839.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv").drop("id", 1)


# The notebook sections below study with training data included.
# The top if statement should prevent the code from running when the public or private test data is presented as both are larger than 200 entries.

In [49]:
if submission.shape[0] == 200:
    print(submission.tail())
    print(submission.shape)

    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction of ensemble scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

      id prediction
195  195      C B A
196  196      B C A
197  197      B A D
198  198      D A C
199  199      C A E
(200, 2)
Prediction of ensemble scored 0.9833333333100001
It got it right on answer 1 a total of 576 times.
It got it right on answer 2 a total of 18 times.
It got it right on answer 3 a total of 6 times.
It missed a total of 0 times.


In [50]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission1.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission1.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission1.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 1 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 1 scored 0.9433333332850007
It got it right on answer 1 a total of 537 times.
It got it right on answer 2 a total of 36 times.
It got it right on answer 3 a total of 15 times.
It missed a total of 12 times.


In [51]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission2.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission2.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission2.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 2 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 2 scored 0.9133333332400012
It got it right on answer 1 a total of 492 times.
It got it right on answer 2 a total of 72 times.
It got it right on answer 3 a total of 24 times.
It missed a total of 12 times.


In [52]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission3.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission3.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission3.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 3 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 3 scored 0.8833333332250013
It got it right on answer 1 a total of 465 times.
It got it right on answer 2 a total of 81 times.
It got it right on answer 3 a total of 33 times.
It missed a total of 21 times.


In [53]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission4.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission4.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission4.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 4 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 4 scored 0.8966666665550019
It got it right on answer 1 a total of 471 times.
It got it right on answer 2 a total of 87 times.
It got it right on answer 3 a total of 27 times.
It missed a total of 15 times.


In [54]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission5.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission5.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission5.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 5 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 5 scored 0.8849999999100011
It got it right on answer 1 a total of 477 times.
It got it right on answer 2 a total of 66 times.
It got it right on answer 3 a total of 30 times.
It missed a total of 27 times.


In [55]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission6.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission6.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission6.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 6 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 6 scored 0.9683333332900005
It got it right on answer 1 a total of 555 times.
It got it right on answer 2 a total of 33 times.
It got it right on answer 3 a total of 12 times.
It missed a total of 0 times.


In [56]:
if submission.shape[0] == 200:
    anscnt1 = 0
    anscnt2 = 0
    anscnt3 = 0
    missedcnt = 0
    score = 0
    for i in range(len(submission)):
        for r in range(3):
            if train_df.loc[i,'answer'] == submission7.loc[i,'prediction'][0]:
                anscnt1 += 1
                score += 1
            elif train_df.loc[i,'answer'] == submission7.loc[i,'prediction'][2]:   
                anscnt2 += 1
                score += .666666666
            elif train_df.loc[i,'answer'] == submission7.loc[i,'prediction'][4]:   
                anscnt3 += 1
                score += .333333333
            else:
                missedcnt += 1
    print ("Prediction 7 scored",score/(3*len(submission)))
    print ("It got it right on answer 1 a total of",anscnt1,"times.")
    print ("It got it right on answer 2 a total of",anscnt2,"times.")
    print ("It got it right on answer 3 a total of",anscnt3,"times.")
    print ("It missed a total of", missedcnt,"times.")

Prediction 7 scored 0.9783333333000002
It got it right on answer 1 a total of 567 times.
It got it right on answer 2 a total of 27 times.
It got it right on answer 3 a total of 6 times.
It missed a total of 0 times.


In [57]:
if submission.shape[0] == 200:
    for r in range(3):
        print ("**** Answer column",r)
        idx = r
        if r == 1: 
            idx = 2
        if r == 2:
            idx = 4
        # Rows represent observer 1's ratings, and columns represent observer 2's ratings
        confusion_matrix = np.array([
            [0, 0, 0, 0, 0],  
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0]])
        for i in range(len(submission)):
            o1idx = ord(train_df.loc[i,'answer'])-ord("A")
            o2idx = ord(submission.loc[i,'prediction'][idx])-ord("A")
            confusion_matrix[o1idx,o2idx] = confusion_matrix[o1idx,o2idx]+1

        # Calculate observed agreement (Po)
        print(confusion_matrix)
        total_ratings = np.sum(confusion_matrix)
        observed_agreement = np.trace(confusion_matrix) / total_ratings

        # Calculate the expected agreement (Pe)
        observer_1_sum = np.sum(confusion_matrix, axis=1)
        observer_2_sum = np.sum(confusion_matrix, axis=0)
        expected_agreement = np.sum(observer_1_sum * observer_2_sum) / (total_ratings ** 2)

        # Calculate Cohen's Kappa
        kappa = (observed_agreement - expected_agreement) / (1 - expected_agreement)

        print(f"Cohen's Kappa: {kappa:.4f}")

**** Answer column 0
[[37  0  0  0  0]
 [ 0 47  1  0  0]
 [ 1  0 43  0  0]
 [ 0  0  0 37  1]
 [ 2  2  1  0 28]]
Cohen's Kappa: 0.9497
**** Answer column 1
[[ 0  9  7 12  9]
 [13  0  6 13 16]
 [13 11  1  8 11]
 [ 7 12 11  1  7]
 [10  6  6  7  4]]
Cohen's Kappa: -0.2083
**** Answer column 2
[[ 0 10  9  9  9]
 [ 8  1 16 15  8]
 [11 10  0 13 10]
 [10  7 12  0  9]
 [ 8  9  6  9  1]]
Cohen's Kappa: -0.2377
